## 1. C# Language Idioms
The C# language has evolved since its first commercial release in 2001. It’s now much farther removed from C++ or Java than it was in its original release.

### Item 1: Prefer Implicitly Typed Local Variables
I prefer var and use it to declare many local variables because I find that it focuses the developer’s attention on the important part (the semantic meaning) and not on the particulars of a variable’s type. The compiler still warns me if I have created any construct that doesn’t type-check.

In [6]:
class MyClass {}

MyClass obj1 = new MyClass();
var obj2 = new MyClass();

In [10]:
Console.WriteLine(obj1.GetType().Name);

MyClass


In [9]:
Console.WriteLine(obj2.GetType().Name);

MyClass


Sometimes using var and not knowing the return type of some function can make things confusing when reading the code.

In [12]:
class GuessWho {
    public static double GetMagicNumber() {
        return 1.0;
    }
}

var num = GuessWho.GetMagicNumber(); // hmmm, int, float, double???
Console.WriteLine(num);
Console.WriteLine(num.GetType().Name);

1
Double


At other times you can shoot yourself in the foot by being type explicit in variable declarations.

For example, [IQueryable<T>](https://docs.microsoft.com/en-us/dotnet/api/system.linq.iqueryable?view=net-5.0) derives from [IEnumerable<T>](https://docs.microsoft.com/en-us/dotnet/api/system.collections.ienumerable?view=net-5.0). In the below example the second query does not call Queryable.Where; instead, it calls Enumerable.Where. That has a large negative implication for performance.

In [46]:
using System;
using System.Collections.Generic;


class Customer {
    public readonly String ContactName;
    
    public Customer(String ContactName) {
        this.ContactName = ContactName;
    } 
}


class DB {
    public IQueryable<Customer> Customers;
    
    public DB() {
        List<Customer> list = new List<Customer>();
        list.Add(new Customer("Bob"));
        list.Add(new Customer("Jane"));
        list.Add(new Customer("Jim"));        
        this.Customers = list.AsQueryable();
    }
}


class BadlyTypedQuery {
    private DB db;
    
    public BadlyTypedQuery() {
        this.db = new DB();
    }

    public IEnumerable<string> FindCustomersStartingWith1(String start) {
        IEnumerable<string> q = 
            from c in db.Customers
            select c.ContactName;
        
        var q2 = q.Where(s => s.StartsWith(start));
        return q2;
    }
    
    // Same method, but different return type.
    public IQueryable<string> FindCustomersStartingWith2(String start) {
        var q = from c in db.Customers select c.ContactName;
        var q2 = q.Where(s => s.StartsWith(start));
        return q2;
    }
}


var query = new BadlyTypedQuery();

var result1 = query.FindCustomersStartingWith1("J");
foreach(String item in result1) {
    Console.WriteLine(item);
}
Console.WriteLine(result1.GetType().Name);
// Jane, Jim
// WhereEnumerableIterator`1


var result2 = query.FindCustomersStartingWith2("J");
foreach(String item in result2) {
    Console.WriteLine(item);
}
Console.WriteLine(result2.GetType().Name);
// Jane, Jim
// EnumerableQuery`1

Jane
Jim
WhereEnumerableIterator`1
Jane
Jim
EnumerableQuery`1


You, the developer, must decide whether letting the compiler silently declare the compile-time type of the variable harms readability. If not being able to immediately see the exact type of a local variable creates ambiguity when someone reads the code, it’s best to declare that type explicitly. However, in many cases, the code clearly conveys the semantic information about the variable.

### Item 2: Prefer readonly to const
You declare runtime constants with the readonly keyword. Compile-time constants are declared with the const keyword. Compile-time constants are slightly faster, but far less flexible.

In [49]:
// Complile-time constant
const int Millennium = 2000;

// Runtime constant
static readonly int ThisYear = 2021;

Console.WriteLine(Millennium);
Console.WriteLine(ThisYear);

2000
2021


In [50]:
Console.WriteLine(Millennium.GetType().Name);

Int32


In [51]:
Console.WriteLine(ThisYear.GetType().Name);

Int32


A compile-time constant is replaced with the value of that constant in your object code. This distinction places several restrictions on when you are allowed to use either type of constant. Compile-time constants can be used only for the built-in integral and floating-point types, enums, or strings.

Read-only values are also constants in that they cannot be modified after the constructor has executed. But read-only values are different in that they are assigned at runtime. You have much more flexibility when working with runtime constants. For one thing, runtime constants can be any type.

In [52]:
class ReadOnlyExample {
    public readonly int num;
    
    public ReadOnlyExample() {
        this.num= 7;
    }
}

var roe = new ReadOnlyExample();

In [54]:
roe.num

7

In [57]:
roe.num = 8; // not after the constructor has run

Unhandled exception: (1,1): error CS0191: A readonly field cannot be assigned to (except in a constructor or a variable initializer)

### Item 3: Prefer the is or as Operators to Casts
The correct choice is to use the as operator whenever you can because it is safer than blindly casting and is more efficient at runtime.

In [61]:
// one way to do it
class MyType {}
class NotMyType {}

class Factory {
    public static MyType GetObject() {
        return new MyType();
    }
}

object obj = Factory.GetObject();
MyType mt = obj as MyType;
if (mt != null) {
    Console.WriteLine("Success casting " + mt.GetType().Name);
} else {
    Console.WriteLine("No luck in casting obj!");
}

Success casting MyType


In [63]:
object obj = new NotMyType();
MyType mt = obj as MyType;
if (mt != null) {
    Console.WriteLine("Success casting " + mt.GetType().Name);
} else {
    Console.WriteLine("No luck in casting obj!");
}

No luck in casting obj!


In [64]:
// another way to cast
object obj = Factory.GetObject();
try {
    MyType mt;
    mt = (MyType)obj;
    Console.WriteLine("Success casting " + mt.GetType().Name);
} catch (InvalidCastException) {
    Console.WriteLine("No luck in casting obj!");
}

Success casting MyType


You’ll agree that the first version is simpler and easier to read. It does not have the try/catch clause, so you avoid both the overhead and the code. Notice that the cast version must check null in addition to catching exceptions.

as and is operators examine the runtime type of the object being converted; they do not perform any other operations, except boxing when necessary. If a particular object is not the requested type or is derived from the requested type, they fail. Casts, on the other hand, can use conversion operators to convert an object to the requested type. This includes any built-in numeric conversions. Casting a long to a short can lose information.

In [73]:
// version one
class SecondType {
    private MyType _value;
    
    public static implicit operator MyType(SecondType t) {
        return t._value;
    }
}


class Factory {
    public static SecondType GetObject() {
        return new SecondType();
    }
}


object obj = Factory.GetObject(); // obj is a SecondType
MyType mt = obj as MyType; // Fails
if (mt != null) {
    Console.WriteLine("Conversion success...");
} else {
    Console.WriteLine("Conversion failed!");
}

Conversion failed!


In [75]:
// version two
try {
    MyType mt;
    mt = (MyType)obj; // Fails
    Console.WriteLine("Conversion success...");
} catch (InvalidCastException) {
    Console.WriteLine("Conversion failed!");
}

Conversion failed!


Both versions fail. It fails because your compiler is generating code based on the compile-time type of the object, o. The compiler knows nothing about the runtime type of o; it views o as an instance of object.

The foreach statement needs to use casts to support both value types and reference types. By choosing the cast operator, the foreach statement exhibits the same behavior, no matter what the destination type is. However, because a cast is used, foreach loops can cause an InvalidCastException to be thrown.

In [91]:
List<object> objects = new List<object>();
objects.Add(1);
objects.Add(2);
objects.Add(3);
objects.Add("4");

foreach(int i in objects) {
    Console.WriteLine(i.ToString() + " " + i.GetType().Name);
}

1 Int32
2 Int32
3 Int32


Unhandled exception: System.InvalidCastException: Unable to cast object of type 'System.String' to type 'System.Int32'.
   at Submission#94.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location where exception was thrown ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [97]:
// to get around this...
foreach(object obj in objects) {
    try {
        int i;
        i = (int)obj;
        Console.WriteLine(i + " " + i.GetType().Name);
    } catch (InvalidCastException) {
        Console.WriteLine("Unable to cast " + obj.GetType().Name + " to int -> " + obj);
    }
}

1 Int32
2 Int32
3 Int32
Unable to cast String to int -> 4


### Item 4: Replace string.Format() with [Interpolated Strings](https://docs.microsoft.com/en-us/dotnet/csharp/language-reference/tokens/interpolated)

In [1]:
Console.WriteLine($"The value of pi is {Math.PI}");

The value of pi is 3.141592653589793


In [2]:
Console.WriteLine($"The value of pi is {Math.PI.ToString()}");

The value of pi is 3.141592653589793


In [3]:
Console.WriteLine($"The value of pi is {Math.PI.ToString("F2")}");

The value of pi is 3.14


In [4]:
Console.WriteLine($"The value of pi is {Math.PI:F2}")

The value of pi is 3.14


### Item 5: Prefer [FormattableString](https://docs.microsoft.com/en-us/dotnet/api/system.formattablestring?view=net-5.0) for Culture-Specific Strings

In [8]:
String first = $"It's the {DateTime.Now.Day} of the {DateTime.Now.Month} month";
Console.WriteLine(first);

It's the 18 of the 5 month


In [9]:
FormattableString second = $"It's the {DateTime.Now.Day} of the {DateTime.Now.Month} month";
Console.WriteLine(second);

It's the 18 of the 5 month


In [10]:
var third = $"It’s the {DateTime.Now.Day} of the {DateTime.Now.Month} month";
Console.WriteLine(third);

It’s the 18 of the 5 month


In [11]:
class CultureSpecificStringsExample {

    public static string ToGerman(FormattableString src) {
        return string.Format(
            null,
            System.Globalization.CultureInfo.CreateSpecificCulture("de-de"),
            src.Format,
            src.GetArguments()
        );
    }

    public static string ToFrenchCanada(FormattableString src) {
        return string.Format(
            null,
            System.Globalization.CultureInfo.CreateSpecificCulture("de-CA"),
            src.Format,
            src.GetArguments()
        );
    }
}

If you run the code in the United States, the decimal separator for a double value will be "."; if you ran the same code in most European countries, the decimal separator would be ",".

### Item 6: Avoid String-ly Typed APIs

### Item 7: Express Callbacks with Delegates
A delegate is a reference type. But instead of referring to an object, a delegate refers to a method.

In [26]:
List<int> numbers = Enumerable.Range(1, 10).ToList();
numbers.RemoveAll(n => n % 2 == 0);
numbers.ForEach(item => Console.WriteLine(item));

1
3
5
7
9


In [27]:
var test = numbers.TrueForAll(n => n < 50);
Console.WriteLine(test);

True


You’ll find numerous examples of this concept in the .NET Framework. All of LINQ is built on delegates. Callbacks are used to handle cross-thread marshalling in Windows Presentation Foundation (WPF) and Windows Forms. Everywhere that the .NET Framework needs a single method, it will use a delegate that callers can express in the form of a lambda expression.

In [31]:
// https://zetcode.com/lang/csharp/delegates/
using System;

delegate void MyDelegate();
var md = new MyDelegate(MyCallback);
md();

void MyCallback() {
    Console.WriteLine("Calling callback");
}


Calling callback


In [32]:
// a delegate can point to different methods over time
using System;


public class Person {
    public string firstName;
    public string secondName;

    public Person(string firstName, string secondName) {
        this.firstName = firstName;
        this.secondName = secondName;
    }

    public void ShowFirstName(string msg) {
        Console.WriteLine($"{msg} {this.firstName}");
    }

    public void ShowSecondName(string msg) {
        Console.WriteLine($"{msg} {this.secondName}");
    }
}

public delegate void NameDelegate(string msg);

var per = new Person("Fabius", "Maximus");
var nDelegate = new NameDelegate(per.ShowFirstName);
nDelegate("Call 1:");

nDelegate = new NameDelegate(per.ShowSecondName);
nDelegate("Call 2:");


Call 1: Fabius
Call 2: Maximus


Multicast delegate is a delegate which holds a reference to more than one method. Multicast delegates must contain only methods that return void, else there is a run-time exception.

In [33]:
using System;


public class Oper {
    public static void Add(int x, int y) {
        Console.WriteLine("{0} + {1} = {2}", x, y, x + y);
    }

    public static void Sub(int x, int y) {
        Console.WriteLine("{0} - {1} = {2}", x, y, x - y);
    }
}


delegate void MyDelegate(int x, int y);

var del = new MyDelegate(Oper.Add);
del += new MyDelegate(Oper.Sub);
del(6, 4);

del -= new MyDelegate(Oper.Sub);
del(2, 8);


6 + 4 = 10
6 - 4 = 2
2 + 8 = 10


In [1]:
// delegates as method parameters
using System;

delegate int Arithm(int x, int y);

void DoOperation(int x, int y, Arithm del) {
    int z = del(x, y);
    Console.WriteLine(z);
}

int Multiply(int x, int y) {
    return x * y;
}

int Divide(int x, int y) {
    return x / y;
}

DoOperation(10, 2, Multiply);
DoOperation(10, 2, Divide);

20
5


Events are messages triggered by some action. A click on a button or a tick of a clock are such actions. The object that triggers an event is called a sender and the object that receives the event is called a receiver.

By convention, event delegates in the .NET Framework have two parameters: the source that raised the event and the data for the event.

In [4]:
using System;


public delegate void OnFiveHandler(object sender, EventArgs e);


class FEvent {
    public event OnFiveHandler FiveEvent;

    public void OnFiveEvent() {
        if (FiveEvent != null) {
            FiveEvent(this, EventArgs.Empty);
        }
    }
}


void Callback(object sender, EventArgs e) {
    Console.WriteLine("Five Event occurred");
}


var fe = new FEvent();
fe.FiveEvent += new OnFiveHandler(Callback);

var random = new Random();

for (int i = 0; i < 10; i++) {
    int rn = random.Next(6);
    Console.WriteLine(rn);

    if (rn == 5){
        fe.OnFiveEvent();
    }
}


3
3
3
5
Five Event occurred
4
5
Five Event occurred
2
3
1
2


### Item 8: Use the Null Conditional Operator for Event Invocations
What if there are no handlers attached to an event? There could even be race conditions involving the code that checks for an attached event handler and the code that invokes it. The new null conditional operator, introduced in C# 6.0, provides a much cleaner syntax for this idiom. You should change your existing habits to use the new syntax as soon as you can.

In [9]:
class EventSource {
    private EventHandler<int> Updated;
    private int counter;
    
    public void RaiseUpdates() {
        counter++;
        Updated(this, counter);
    }
}


var es = new EventSource();
// es.RaiseUpdates() // System.NullReferenceException

This code has an obvious issue. If this object runs when no event handlers have been attached to the Updated event, the code will throw a NullReferenceException. If we take a step in the right direction...

In [10]:
class EventSource {
    private EventHandler<int> Updated;
    private int counter;
    
    public void RaiseUpdates() {
        counter++;
        if (Updated != null) {
            Updated(this, counter);
        }
    }
}

it’s possible that another thread will execute and unsubscribe the single event handler. When the first thread continues execution and invokes the event handler, the handler is null. You’ll still get that NullReferenceException. However, you’ll get it more rarely, and not in easy-to-reproduce scenarios.

Experienced developers have learned from harsh experience that this idiom is dangerous and have replaced it with this code:

In [11]:
class EventSource {
    private EventHandler<int> Updated;
    private int counter;
    
    public void RaiseUpdates() {
        counter++;
        var handler = Updated;
        if (handler != null) {
            handler(this, counter);
        }
    }
}

This code has been the recommended practice for raising events in .NET and C#. It works, and it’s thread safe. But it has several issues in terms of readability. It works, but it’s not easy for new .NET developers to see and understand. Also, it must be replicated everywhere the event is raised.

The null conditional operator restores this action to a much simpler code construct.

In [12]:
class EventSource {
    private EventHandler<int> Updated;
    private int counter;
    
    public void RaiseUpdates() {
        counter++;
        Updated?.Invoke(this, counter);
    }
}

This code uses the null conditional operator ("?.") to safely invoke the event handlers. The "?." operator evaluates the left-hand side of the operator. If that evaluates to a non-null value, the expression on the right-hand side of the operator is executed. If the expression is null, it short-circuits and execution continues at the next statement.

This code is thread safe. It’s also more concise.

### Item 9: Minimize Boxing and Unboxing
Boxing places a value type in an untyped reference object to allow the value type to be used where a reference type is expected. Unboxing extracts a copy of that value type from the box. Boxing and unboxing are necessary for you to use value types where the System.Object type or an interface type is expected. But boxing and unboxing are always performance-robbing operations.

Watch for implicit conversions to System.Object. Value types should not be substituted for System.Object if you can avoid it.

### Item 10: Use the new Modifier Only to React to Base Class Updates
You can use the new modifier on a class member to redefine a nonvirtual member inherited from a base class. Just because you can do something doesn’t mean you should, though. Redefining nonvirtual methods creates ambiguous behavior.

In [16]:
class MyClass {
    public void MagicMethod() {
        Console.WriteLine("MyClass");
    }
}


class MyOtherClass {
    public new void MagicMethod() {
        Console.WriteLine("MyOtherClass");
    }
}


class Helper {
    public static object MakeObject() {
        return new MyOtherClass();
    }
}

object c = Helper.MakeObject();
Console.WriteLine(c.GetType().Name);

MyOtherClass


In [20]:
MyOtherClass moc = c as MyOtherClass;
Console.WriteLine(moc.GetType().Name);

MyOtherClass


In [26]:
MyClass mc = c as MyClass;
Console.WriteLine(mc is null);

True


Nonvirtual methods are statically bound. Any source code anywhere that references MyClass.MagicMethod() calls exactly that function. Nothing in the runtime looks for a different version defined in any derived classes.

The recommendation to avoid using the new modifier to redefine nonvirtual functions should not be interpreted as a recommendation to make everything virtual when you define base classes.

***